In [ ]:
"""
===============================================================================
LEGO Status Build Analysis - Report e Gestione Inventari XML
===============================================================================

Questo script consente di:
- Generare report PDF dettagliati sull'avanzamento della collezione LEGO, analizzando i file inventario XML esportati da BrickLink.
- Combinare e filtrare più file XML in un unico file per la gestione degli ordini di parti mancanti.

Funzionalità principali:
------------------------
1. Analisi e report PDF:
   - Estrae quantità minime, quantità possedute e totali per ogni colore e set.
   - Genera grafici a barre e a torta per ogni set e per la collezione complessiva.
   - Evidenzia eventuali codici colore mancanti nella mappatura.
   - Salva il report in un unico file PDF multi-pagina.

2. Combinazione e filtro XML:
   - Esclude file XML specificati dall’analisi.
   - Filtra solo gli elementi con quantità minima > 0.
   - Combina gli elementi uguali (stesso tipo e colore) sommando le quantità.
   - Crea file XML filtrati per ogni inventario e un file XML combinato per l’ordine.

Struttura del codice:
---------------------
- Classe `LegoColorReport`: gestisce la generazione del report PDF.
- Classe `LegoXmlCombiner`: gestisce la combinazione e il filtro dei file XML.
- Sezione `if __name__ == "__main__"`: configura e avvia i processi di report e combinazione.

Dipendenze:
-----------
- Python 3.x
- matplotlib
- xml.etree.ElementTree
- collections
- json
- os

Utilizzo:
---------
1. Aggiorna i percorsi delle cartelle e dei file secondo la tua struttura locale.
2. Assicurati che il file di mappatura colori JSON sia presente e aggiornato.
3. Esegui lo script per generare i report PDF e i file XML combinati/filtrati.

Output:
-------
- Report PDF con grafici e riepiloghi per ogni set e per la collezione.
- File XML filtrati e combinati pronti per l’importazione su BrickLink.

===============================================================================
"""

import os
import json
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.backends.backend_pdf import PdfPages

class LegoColorReport:
    def __init__(self, folder_path, color_mapping_path, output_pdf):
        self.folder_path = folder_path
        self.color_mapping_path = color_mapping_path
        self.output_pdf = output_pdf
        self.color_mapping = self.load_color_mapping()
        self.xml_files = [f for f in os.listdir(self.folder_path) if f.endswith('.xml')]
        self.all_warnings = []
        self.overall_status = []
        self.parts_count = []
        self.parts_count_owned = []
        self.total_min_qty = 0
        self.total_qty_filled = 0
        self.overall_color_distribution = defaultdict(int)
        self.overall_color_filled_distribution = defaultdict(int)

    def load_color_mapping(self):
        with open(self.color_mapping_path, 'r', encoding='utf-8') as f:
            return json.load(f)

    def process(self):
        print("XML files found:", self.xml_files)
        with PdfPages(self.output_pdf) as pdf:
            for xml_file in self.xml_files:
                self.process_single_file(xml_file, pdf)
            self.add_overall_summary(pdf)
            self.add_warnings_page(pdf)
        print(f"Report saved to {self.output_pdf}")

    def process_single_file(self, xml_file, pdf):
        file_path = os.path.join(self.folder_path, xml_file)
        min_qty_data = defaultdict(int)
        qty_filled_data = defaultdict(int)
        total_qty_data = defaultdict(int)
        set_min_qty = 0
        set_qty_filled = 0

        tree = ET.parse(file_path)
        root = tree.getroot()

        for item in root.findall('ITEM'):
            color = self.get_xml_text(item, 'COLOR', default='0', numeric=True)
            min_qty = self.get_xml_int(item, 'MINQTY')
            qty_filled = self.get_xml_int(item, 'QTYFILLED')
            item_id = self.get_xml_text(item, 'ITEMID', default='Unknown ID')

            total_qty = min_qty + qty_filled
            min_qty_data[color] += min_qty
            qty_filled_data[color] += qty_filled
            total_qty_data[color] += total_qty
            self.overall_color_distribution[color] += total_qty
            self.overall_color_filled_distribution[color] += qty_filled
            set_min_qty += min_qty
            set_qty_filled += qty_filled

        set_completion_status = (set_qty_filled / (set_qty_filled + set_min_qty)) * 100 if (set_qty_filled + set_min_qty) > 0 else 0
        self.overall_status.append((xml_file, set_completion_status))
        self.parts_count.append((xml_file, set_qty_filled + set_min_qty))
        self.parts_count_owned.append((xml_file, set_qty_filled))
        self.total_min_qty += set_min_qty
        self.total_qty_filled += set_qty_filled

        self.plot_set_chart(
            xml_file, min_qty_data, qty_filled_data, total_qty_data,
            set_min_qty, set_qty_filled, pdf
        )

    def plot_set_chart(self, xml_file, min_qty_data, qty_filled_data, total_qty_data, set_min_qty, set_qty_filled, pdf):
        colors = list(min_qty_data.keys())
        min_qty_values = [min_qty_data[c] for c in colors]
        qty_filled_values = [qty_filled_data[c] for c in colors]
        total_qty_values = [total_qty_data[c] for c in colors]

        # Sort by total_qty_values descending
        sorted_indices = sorted(range(len(total_qty_values)), key=lambda k: total_qty_values[k], reverse=True)
        colors = [colors[i] for i in sorted_indices]
        min_qty_values = [min_qty_values[i] for i in sorted_indices]
        qty_filled_values = [qty_filled_values[i] for i in sorted_indices]
        total_qty_values = [total_qty_values[i] for i in sorted_indices]

        color_names = []
        for color in colors:
            if color in self.color_mapping:
                color_names.append(self.color_mapping[color])
            else:
                color_names.append(color)
                warning = f"Warning: Color code {color} not found in color mapping. Using color code as label. File: {xml_file}"
                self.all_warnings.append(warning)

        x = range(len(colors))
        plt.figure(figsize=(14, 8))
        bar_width = 0.25
        plt.bar(x, min_qty_values, width=bar_width, label='Min Qty', align='center', color='#0072B2')
        plt.bar([p + bar_width for p in x], qty_filled_values, width=bar_width, label='Qty Filled', align='center', color='#E69F00')
        plt.bar([p + bar_width * 2 for p in x], total_qty_values, width=bar_width, label='Total Qty', align='center', color='#009E73')
        plt.xlabel('Color')
        plt.ylabel('Quantity')
        plt.title(f'LEGO Set Color Quantities - {xml_file}')
        plt.xticks([p + bar_width for p in x], color_names, rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()

        # Pie chart inset
        inset_ax = plt.axes([0.65, 0.55, 0.3, 0.3])
        labels = ['Qty Filled', 'Qty Not Filled']
        sizes = [set_qty_filled, set_min_qty]
        pie_colors = ['#ff9999', '#66b3ff']
        explode = (0.1, 0)
        inset_ax.pie(sizes, explode=explode, labels=labels, colors=pie_colors, autopct='%1.1f%%',
                     shadow=True, startangle=140)
        inset_ax.axis('equal')
        inset_ax.set_title('Completion')
        pdf.savefig()
        plt.close()

    def add_overall_summary(self, pdf):
        overall_completion_status = (self.total_qty_filled / (self.total_qty_filled + self.total_min_qty)) * 100 if (self.total_qty_filled + self.total_min_qty) > 0 else 0
        total_bricks = sum(self.overall_color_distribution.values())
        total_bricks_owned = sum(self.overall_color_filled_distribution.values())
        combined_text = "\n".join([
            f"{xml_file}: {status:.2f}% - {count} bricks"
            for (xml_file, status), (_, count) in zip(self.overall_status, self.parts_count)
        ])

        plt.figure(figsize=(8, 14))
        plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
        plt.text(0.1, 0.95, f'Overall Collection Completion Status: {overall_completion_status:.2f}%', ha='left', va='top', fontsize=14, wrap=True)
        plt.text(0.1, 0.9, f'Overall number of Bricks: {total_bricks:.0f}', ha='left', va='top', fontsize=14, wrap=True)
        plt.text(0.1, 0.85, f'Overall number of Bricks Owned: {total_bricks_owned:.0f}', ha='left', va='top', fontsize=14, wrap=True)
        plt.text(0.1, 0.8, "Set Completion Status:\n" + combined_text, ha='left', va='top', fontsize=11, wrap=True)
        plt.axis('off')
        pdf.savefig()
        plt.close()

        # Overall color distribution
        overall_colors = list(self.overall_color_distribution.keys())
        overall_color_values = [self.overall_color_distribution[c] for c in overall_colors]
        overall_color_filled_values = [self.overall_color_filled_distribution[c] for c in overall_colors]
        overall_color_names = [self.color_mapping.get(c, c) for c in overall_colors]

        sorted_indices = sorted(range(len(overall_color_values)), key=lambda k: overall_color_values[k], reverse=True)
        overall_colors = [overall_colors[i] for i in sorted_indices]
        overall_color_values = [overall_color_values[i] for i in sorted_indices]
        overall_color_filled_values = [overall_color_filled_values[i] for i in sorted_indices]
        overall_color_names = [overall_color_names[i] for i in sorted_indices]

        plt.figure(figsize=(14, 8))
        bar_width = 0.35
        plt.bar(range(len(overall_colors)), overall_color_values, width=bar_width, label='Total Qty', align='center', color='#009E73')
        plt.bar([p + bar_width for p in range(len(overall_colors))], overall_color_filled_values, width=bar_width, label='Qty Filled', align='center', color='#E69F00')
        plt.xlabel('Color')
        plt.ylabel('Total Quantity')
        plt.title('Overall Color Distribution')
        plt.xticks([p + bar_width / 2 for p in range(len(overall_colors))], overall_color_names, rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()

        # Pie chart inset
        inset_ax = plt.axes([0.65, 0.55, 0.3, 0.3])
        labels = ['Qty Filled', 'Qty Not Filled']
        sizes = [self.total_qty_filled, self.total_min_qty]
        pie_colors = ['#ff9999', '#66b3ff']
        explode = (0.1, 0)
        inset_ax.pie(sizes, explode=explode, labels=labels, colors=pie_colors, autopct='%1.1f%%',
                     shadow=True, startangle=140)
        inset_ax.axis('equal')
        inset_ax.set_title('Overall Completion')
        pdf.savefig()
        plt.close()

    def add_warnings_page(self, pdf):
        unique_warnings = list(set(self.all_warnings))
        if unique_warnings:
            plt.figure(figsize=(14, 8))
            plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
            plt.text(0.5, 0.9, "Warnings:", ha='center', va='top', fontsize=14, wrap=True)
            plt.text(0.1, 0.8, "\n".join(unique_warnings), ha='left', va='top', fontsize=10, wrap=True)
            plt.axis('off')
            pdf.savefig()
            plt.close()

    @staticmethod
    def get_xml_text(item, tag, default='', numeric=False):
        el = item.find(tag)
        if el is not None and el.text:
            if numeric:
                return el.text if el.text.isdigit() else default
            return el.text
        return default

    @staticmethod
    def get_xml_int(item, tag):
        el = item.find(tag)
        return int(el.text) if el is not None and el.text and el.text.isdigit() else 0

class LegoXmlCombiner:
    def __init__(self, folder_path, filtered_folder, output_file, excluded_files=None):
        self.folder_path = folder_path
        self.filtered_folder = filtered_folder
        self.output_file = output_file
        self.excluded_files = excluded_files if excluded_files else []
        os.makedirs(self.filtered_folder, exist_ok=True)
        self.xml_files = [f for f in os.listdir(self.folder_path) if f.endswith('.xml') and f not in self.excluded_files]
        self.combined_root = ET.Element("INVENTORY")
        self.item_tracker = {}

    def process(self):
        print("XML files found (excluding specified files):", self.xml_files)
        for xml_file in self.xml_files:
            self.process_single_file(xml_file)
        self.write_combined_xml()

    def process_single_file(self, xml_file):
        file_path = os.path.join(self.folder_path, xml_file)
        tree = ET.parse(file_path)
        root = tree.getroot()
        filtered_root = ET.Element(root.tag)

        for item in root.findall('ITEM'):
            min_qty = self.get_xml_int(item, 'MINQTY')
            item_type = self.get_xml_text(item, 'ITEMID')
            color = self.get_xml_text(item, 'COLOR')

            if min_qty > 0 and item_type and color:
                item_key = (item_type, color)
                # Combined file logic
                if item_key in self.item_tracker:
                    existing_item = self.item_tracker[item_key]
                    existing_min_qty = int(existing_item.find('MINQTY').text)
                    existing_item.find('MINQTY').text = str(existing_min_qty + min_qty)
                else:
                    new_combined_item = self.copy_item(item, xml_file)
                    self.combined_root.append(new_combined_item)
                    self.item_tracker[item_key] = new_combined_item
                # Filtered file logic
                new_filtered_item = self.copy_item(item)
                filtered_root.append(new_filtered_item)

        filtered_file_path = os.path.join(self.filtered_folder, f"filtered_{xml_file}")
        filtered_tree = ET.ElementTree(filtered_root)
        filtered_tree.write(filtered_file_path, encoding='utf-8', xml_declaration=True)

    def write_combined_xml(self):
        combined_tree = ET.ElementTree(self.combined_root)
        combined_tree.write(self.output_file, encoding='utf-8', xml_declaration=True)
        print(f"Combined wanted list XML file created: {self.output_file}")

    @staticmethod
    def copy_item(item, remarks=None):
        new_item = ET.Element('ITEM')
        for sub_element in item:
            new_sub_element = ET.SubElement(new_item, sub_element.tag)
            if sub_element.tag == 'QTYFILLED':
                new_sub_element.text = '0'
            else:
                new_sub_element.text = sub_element.text
        if remarks:
            remarks_element = ET.SubElement(new_item, 'REMARKS')
            remarks_element.text = remarks
        return new_item

    @staticmethod
    def get_xml_text(item, tag):
        el = item.find(tag)
        return el.text if el is not None else None

    @staticmethod
    def get_xml_int(item, tag):
        el = item.find(tag)
        return int(el.text) if el is not None and el.text and el.text.isdigit() else 0

# --- USAGE EXAMPLES ---

if __name__ == "__main__":
    # Definizione dei report da generare
    reports = [
        {
            "folder_path": r'C:\Development\BrickLinkReport',
            "color_mapping_path": 'color_mapping.json',
            "output_pdf": os.path.join(r'C:\Development\BrickLinkReport', 'LEGO_Set_Color_Quantities_Report.pdf')
        },
        {
            "folder_path": r'C:\Development\BrickLinkReport_Other',
            "color_mapping_path": 'BL_color_mapping.json',
            "output_pdf": os.path.join(r'C:\Development\BrickLinkReport_Other', 'LEGO_Set_Color_Quantities_Report_NonLOTR.pdf')
        }
    ]

    # Generazione dei report PDF
    for report_cfg in reports:
        report = LegoColorReport(
            folder_path=report_cfg["folder_path"],
            color_mapping_path=report_cfg["color_mapping_path"],
            output_pdf=report_cfg["output_pdf"]
        )
        report.process()

    # Definizione delle combinazioni XML da generare
    combiners = [
        {
            "folder_path": r'C:\Development\BrickLinkReport',
            "filtered_folder": r'C:\Development\BrickLinkReport\Filtered',
            "output_file": os.path.join(r'C:\Development\BrickLinkReport\Filtered', "wanted_list.xml"),
            "excluded_files": [    
                "10333 - Barad-dur.xml",
                "10237 - Tower of Orthanc.xml"
            ]
        },
        {
            "folder_path": r'C:\Development\BrickLinkReport_Other',
            "filtered_folder": r'C:\Development\BrickLinkReport_Other\Filtered',
            "output_file": os.path.join(r'C:\Development\BrickLinkReport_Other\Filtered', "wanted_list_Other.xml"),
            "excluded_files": []
        }
    ]

    # Generazione dei file XML combinati e filtrati
    for combiner_cfg in combiners:
        combiner = LegoXmlCombiner(
            folder_path=combiner_cfg["folder_path"],
            filtered_folder=combiner_cfg["filtered_folder"],
            output_file=combiner_cfg["output_file"],
            excluded_files=combiner_cfg["excluded_files"]
        )
        combiner.process()

BACKUP

In [ ]:
""" No LOTR  -- GENERATION OF LEGO SET COLOR QUANTITIES REPORT
This script generates a detailed PDF report analyzing LEGO set color quantities from XML inventory files. 
It includes bar charts, pie charts, and textual summaries for each set and overall collection.

Key Features:
1. Parses XML files to extract color quantities and completion status for LEGO sets.
2. Generates bar charts for minimum quantity, filled quantity, and total quantity for each set.
3. Adds pie charts to visualize the completion status for each set and overall collection.
4. Produces an overall summary of the collection, including total bricks and completion percentage.
5. Highlights warnings for missing color mappings in the JSON file.

Modules Used:
- xml.etree.ElementTree: For parsing XML files.
- matplotlib.pyplot: For creating bar and pie charts.
- collections.defaultdict: For aggregating data by color.
- json: For loading color mappings.
- matplotlib.backends.backend_pdf.PdfPages: For saving multiple plots to a single PDF.
- os: For file and directory operations.

Variables:
- folder_path: Path to the folder containing the XML files.
- color_mapping: Dictionary mapping color codes to color names, loaded from a JSON file.
- xml_files: List of XML files to process.
- pdf_path: Path to save the generated PDF report.
- overall_color_distribution: Aggregated total quantities for each color across all sets.
- overall_color_filled_distribution: Aggregated filled quantities for each color across all sets.
- total_min_qty, total_qty_filled: Total minimum and filled quantities across all sets.

Workflow:
1. Load color mappings from a JSON file.
2. Scan the folder for XML files and parse each file.
3. Extract color quantities and calculate completion status for each set.
4. Generate bar charts for each set, sorted by total quantity.
5. Add pie charts to visualize completion status for each set and overall collection.
6. Create an overall summary page with textual data and a bar chart for color distribution.
7. Save all plots and warnings to a multi-page PDF report.

Output:
- A PDF report containing:
    - Bar charts for each set's color quantities.
    - Pie charts for each set's completion status.
    - An overall summary page with completion percentage and total bricks.
    - A bar chart for overall color distribution.
    - A page listing warnings for missing color mappings.

Usage:
- Update `folder_path` and `color_mapping.json` to match your directory structure.
- Place XML inventory files in the specified folder.
- Run the script to generate the PDF report.
"""

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from collections import defaultdict
import json
from matplotlib.backends.backend_pdf import PdfPages
import os

# Load the color mapping from the JSON file
with open('BL_color_mapping.json', 'r') as f:
    color_mapping = json.load(f)

# Define the folder containing the XML files
folder_path = r'C:\Development\BrickLinkReport_Other'
# Scan the folder and use all the XML files found
xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

# Print the list of XML files found for debugging
print("XML files found:", xml_files)

# Initialize PdfPages to save the report
pdf_path = os.path.join(folder_path, 'LEGO_Set_Color_Quantities_Report_NonLOTR.pdf')
with PdfPages(pdf_path) as pdf:
    all_warnings = []
    overall_status = []
    parts_count = []
    parts_count_owned = []
    total_min_qty = 0
    total_qty_filled = 0
    total_qty = 0
    total_qty_overall = 0

    overall_color_distribution = defaultdict(int)
    overall_color_filled_distribution = defaultdict(int)

    for xml_file in xml_files:
        # Print the full path of the current XML file for debugging
        file_path = os.path.join(folder_path, xml_file)

        # Initialize dictionaries to store the data
        min_qty_data = defaultdict(int)
        qty_filled_data = defaultdict(int)
        total_qty_data = defaultdict(int)
        warnings = []
        set_min_qty = 0
        set_qty_filled = 0

        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Extract data from XML
        for item in root.findall('ITEM'):
            color_element = item.find('COLOR')
            qty_element = item.find('MINQTY')
            qty_own = item.find('QTYFILLED')
            item_id_element = item.find('ITEMID')
            item_id = item_id_element.text if item_id_element is not None else 'Unknown ID'
            
            color = color_element.text if color_element is not None and color_element.text and color_element.text.isdigit() else '0'
            min_qty = int(qty_element.text) if qty_element is not None and qty_element.text and qty_element.text.isdigit() else 0
            qty_filled = int(qty_own.text) if qty_own is not None and qty_own.text is not None and qty_own.text.isdigit() else 0
            total_qty = min_qty + qty_filled

            min_qty_data[color] += min_qty
            qty_filled_data[color] += qty_filled
            total_qty_data[color] += total_qty
            overall_color_distribution[color] += total_qty
            overall_color_filled_distribution[color] += qty_filled

            set_min_qty += min_qty
            set_qty_filled += qty_filled

        # Calculate completion status for the set
        set_completion_status = (set_qty_filled / (set_qty_filled + set_min_qty)) * 100 
        overall_status.append((xml_file, set_completion_status))
        parts_count.append((xml_file, set_qty_filled + set_min_qty))
        parts_count_owned.append((xml_file, set_qty_filled))

        total_min_qty += set_min_qty
        total_qty_filled += set_qty_filled

        # Prepare data for plotting
        colors = list(min_qty_data.keys())
        min_qty_values = [min_qty_data[color] for color in colors]
        qty_filled_values = [qty_filled_data[color] for color in colors]
        total_qty_values = [total_qty_data[color] for color in colors]

        # Sort by descending order of total_qty_values
        sorted_indices = sorted(range(len(total_qty_values)), key=lambda k: total_qty_values[k], reverse=True)
        colors = [colors[i] for i in sorted_indices]
        min_qty_values = [min_qty_values[i] for i in sorted_indices]
        qty_filled_values = [qty_filled_values[i] for i in sorted_indices]
        total_qty_values = [total_qty_values[i] for i in sorted_indices]

        # Map color codes to color names
        color_names = []
        for color in colors:
            if color in color_mapping:
                color_names.append(color_mapping[color])
            else:
                color_names.append(color)
                warning_message = f"Warning: Color code {color} not found in color mapping. Using color code as label. File: {xml_file}, Item ID: {item_id}"
                warnings.append(warning_message)
                all_warnings.append(warning_message)

        x = range(len(colors))

        # Plot the data
        plt.figure(figsize=(14, 8))
        bar_width = 0.25

        plt.bar(x, min_qty_values, width=bar_width, label='Min Qty', align='center', color='#0072B2')
        plt.bar([p + bar_width for p in x], qty_filled_values, width=bar_width, label='Qty Filled', align='center', color='#E69F00')
        plt.bar([p + bar_width * 2 for p in x], total_qty_values, width=bar_width, label='Total Qty', align='center', color='#009E73')

        plt.xlabel('Color')
        plt.ylabel('Quantity')
        plt.title(f'LEGO Set Color Quantities - {xml_file}')
        plt.xticks([p + bar_width for p in x], color_names, rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()

        # Add the pie chart for qty filled vs not filled in the top right corner
        inset_ax = plt.axes([0.65, 0.55, 0.3, 0.3])  # Position: [left, bottom, width, height]
        labels = ['Qty Filled', 'Qty Not Filled']
        sizes = [set_qty_filled, set_min_qty]
        colors = ['#ff9999','#66b3ff']
        explode = (0.1, 0)  # explode the 1st slice (Qty Filled)

        inset_ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
                     shadow=True, startangle=140)
        inset_ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        inset_ax.set_title('Completion')

        # Save the plot to the PDF file
        pdf.savefig()
        plt.close()

    # Calculate overall completion status
    overall_completion_status = (total_qty_filled / (total_qty_filled + total_min_qty)) * 100
    # Calculate total number of bricks
    total_bricks = sum(overall_color_distribution.values())
    total_bricks_owned = sum(overall_color_filled_distribution.values())
    # Add overall status to the first page of the PDF
    plt.figure(figsize=(8, 14))  # Vertical page
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    plt.text(0.1, 0.95, f'Overall Collection Completion Status: {overall_completion_status:.2f}%', ha='left', va='top', fontsize=14, wrap=True)
    plt.text(0.1, 0.9, f'Overall number of Bricks: {total_bricks:.0f}', ha='left', va='top', fontsize=14, wrap=True)
    plt.text(0.1, 0.85, f'Overall number of Bricks Owned: {total_bricks_owned:.0f}', ha='left', va='top', fontsize=14, wrap=True)
    status_text = "\n".join([f"{xml_file}: {status:.2f}%" for xml_file, status in overall_status])
    parts_count_text = "\n".join([f"{xml_file}: {count} bricks" for xml_file, count in parts_count])
    combined_text = "\n".join([f"{xml_file}: {status:.2f}% - {count} bricks" for (xml_file, status), (_, count) in zip(overall_status, parts_count)])
    plt.text(0.1, 0.8, "Set Completion Status:\n" + combined_text, ha='left', va='top', fontsize=11, wrap=True)
    plt.axis('off')
    pdf.savefig()
    plt.close()

    # Plot overall color distribution
    overall_colors = list(overall_color_distribution.keys())
    overall_color_values = [overall_color_distribution[color] for color in overall_colors]
    overall_color_filled_values = [overall_color_filled_distribution[color] for color in overall_colors]
    overall_color_names = [color_mapping[color] if color in color_mapping else color for color in overall_colors]

    # Sort by descending order
    sorted_indices = sorted(range(len(overall_color_values)), key=lambda k: overall_color_values[k], reverse=True)
    overall_colors = [overall_colors[i] for i in sorted_indices]
    overall_color_values = [overall_color_values[i] for i in sorted_indices]
    overall_color_filled_values = [overall_color_filled_values[i] for i in sorted_indices]
    overall_color_names = [overall_color_names[i] for i in sorted_indices]

    plt.figure(figsize=(14, 8))
    bar_width = 0.35

    plt.bar(range(len(overall_colors)), overall_color_values, width=bar_width, label='Total Qty', align='center', color='#009E73')
    plt.bar([p + bar_width for p in range(len(overall_colors))], overall_color_filled_values, width=bar_width, label='Qty Filled', align='center', color='#E69F00')

    plt.xlabel('Color')
    plt.ylabel('Total Quantity')
    plt.title('Overall Color Distribution')
    plt.xticks([p + bar_width / 2 for p in range(len(overall_colors))], overall_color_names, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()

    # Add the pie chart for overall qty filled vs not filled in the top right corner
    inset_ax = plt.axes([0.65, 0.55, 0.3, 0.3])  # Position: [left, bottom, width, height]
    labels = ['Qty Filled', 'Qty Not Filled']
    sizes = [total_qty_filled, total_min_qty]
    colors = ['#ff9999','#66b3ff']
    explode = (0.1, 0)  # explode the 1st slice (Qty Filled)

    inset_ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
                 shadow=True, startangle=140)
    inset_ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    inset_ax.set_title('Overall Completion')

    # Save the overall color distribution plot to the PDF file
    pdf.savefig()
    plt.close()

    # Remove duplicate warnings
    all_warnings = list(set(all_warnings))

    # Add warnings to the last page of the PDF
    if all_warnings:
        plt.figure(figsize=(14, 8))  # Vertical page
        plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
        plt.text(0.5, 0.9, "Warnings:", ha='center', va='top', fontsize=14, wrap=True)
        plt.text(0.1, 0.8, "\n".join(all_warnings), ha='left', va='top', fontsize=10, wrap=True)
        plt.axis('off')
        pdf.savefig()
        plt.close()


print(f"Report saved to {pdf_path}")

filtered_folder = r'C:\Development\BrickLinkReport_Other\Filtered'
# Define the output file for the combined XML
output_file_path = os.path.join(filtered_folder, "wanted_list_Other.xml")

# List of XML files to exclude
excluded_files = [
]
# Ensure the output folder exists
os.makedirs(filtered_folder, exist_ok=True)

# Scan the folder and use all the XML files found, excluding the specified files
xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml') and f not in excluded_files]

# Print the list of XML files found for debugging
print("XML files found (excluding specified files):", xml_files)

# Create a new root for the combined XML
combined_root = ET.Element("INVENTORY")

# Dictionary to track existing items by (Type, Color)
item_tracker = {}

for xml_file in xml_files:
    # Print the full path of the current XML file for debugging
    file_path = os.path.join(folder_path, xml_file)

    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Create a new root for the filtered XML
    filtered_root = ET.Element(root.tag)

    # Extract data from XML and filter items
    for item in root.findall('ITEM'):
        qty_element = item.find('MINQTY')
        item_id_element = item.find('ITEMID')
        color_element = item.find('COLOR')

        min_qty = int(qty_element.text) if qty_element is not None and qty_element.text and qty_element.text.isdigit() else 0
        item_type = item_id_element.text if item_id_element is not None else None
        color = color_element.text if color_element is not None else None

        if min_qty > 0 and item_type and color:
            # Create a unique key for the item based on Type and Color
            item_key = (item_type, color)

            # Add to the combined file
            if item_key in item_tracker:
                # If the item already exists, add the MINQTY to the existing one
                existing_item = item_tracker[item_key]
                existing_min_qty = int(existing_item.find('MINQTY').text)
                existing_item.find('MINQTY').text = str(existing_min_qty + min_qty)
            else:
                # Create a new ITEM element for the combined file
                new_combined_item = ET.Element('ITEM')

                # Copy all sub-elements from the original item
                for sub_element in item:
                    new_sub_element = ET.SubElement(new_combined_item, sub_element.tag)
                    if sub_element.tag == 'QTYFILLED':
                        # Set QTYFILLED to 0
                        new_sub_element.text = '0'
                    else:
                        # Copy the original text
                        new_sub_element.text = sub_element.text

                # Add the REMARKS field with the name of the original file
                remarks_element = ET.SubElement(new_combined_item, 'REMARKS')
                remarks_element.text = f"{xml_file}"

                # Add the new item to the combined root and tracker
                combined_root.append(new_combined_item)
                item_tracker[item_key] = new_combined_item

            # Add to the filtered file
            new_filtered_item = ET.Element('ITEM')

            # Copy all sub-elements from the original item
            for sub_element in item:
                new_sub_element = ET.SubElement(new_filtered_item, sub_element.tag)
                if sub_element.tag == 'QTYFILLED':
                    # Set QTYFILLED to 0
                    new_sub_element.text = '0'
                else:
                    # Copy the original text
                    new_sub_element.text = sub_element.text

            # Add the new item to the filtered root
            filtered_root.append(new_filtered_item)

    # Write the filtered XML to a file
    filtered_file_path = os.path.join(filtered_folder, f"filtered_{xml_file}")
    filtered_tree = ET.ElementTree(filtered_root)
    filtered_tree.write(filtered_file_path, encoding='utf-8', xml_declaration=True)
    #print(f"Filtered XML file created: {filtered_file_path}")

# Write the combined XML to a file
combined_tree = ET.ElementTree(combined_root)
combined_tree.write(output_file_path, encoding='utf-8', xml_declaration=True)
print(f"Combined wanted list XML file created: {output_file_path}")

In [ ]:
""" GENERATION OF LEGO SET COLOR QUANTITIES REPORT
This script generates a detailed PDF report analyzing LEGO set color quantities from XML inventory files. 
It includes bar charts, pie charts, and textual summaries for each set and overall collection.

Key Features:
1. Parses XML files to extract color quantities and completion status for LEGO sets.
2. Generates bar charts for minimum quantity, filled quantity, and total quantity for each set.
3. Adds pie charts to visualize the completion status for each set and overall collection.
4. Produces an overall summary of the collection, including total bricks and completion percentage.
5. Highlights warnings for missing color mappings in the JSON file.

Modules Used:
- xml.etree.ElementTree: For parsing XML files.
- matplotlib.pyplot: For creating bar and pie charts.
- collections.defaultdict: For aggregating data by color.
- json: For loading color mappings.
- matplotlib.backends.backend_pdf.PdfPages: For saving multiple plots to a single PDF.
- os: For file and directory operations.

Variables:
- folder_path: Path to the folder containing the XML files.
- color_mapping: Dictionary mapping color codes to color names, loaded from a JSON file.
- xml_files: List of XML files to process.
- pdf_path: Path to save the generated PDF report.
- overall_color_distribution: Aggregated total quantities for each color across all sets.
- overall_color_filled_distribution: Aggregated filled quantities for each color across all sets.
- total_min_qty, total_qty_filled: Total minimum and filled quantities across all sets.

Workflow:
1. Load color mappings from a JSON file.
2. Scan the folder for XML files and parse each file.
3. Extract color quantities and calculate completion status for each set.
4. Generate bar charts for each set, sorted by total quantity.
5. Add pie charts to visualize completion status for each set and overall collection.
6. Create an overall summary page with textual data and a bar chart for color distribution.
7. Save all plots and warnings to a multi-page PDF report.

Output:
- A PDF report containing:
    - Bar charts for each set's color quantities.
    - Pie charts for each set's completion status.
    - An overall summary page with completion percentage and total bricks.
    - A bar chart for overall color distribution.
    - A page listing warnings for missing color mappings.

Usage:
- Update `folder_path` and `color_mapping.json` to match your directory structure.
- Place XML inventory files in the specified folder.
- Run the script to generate the PDF report.
"""

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from collections import defaultdict
import json
from matplotlib.backends.backend_pdf import PdfPages
import os

# Load the color mapping from the JSON file
with open('BL_color_mapping.json', 'r') as f:
    color_mapping = json.load(f)

# Define the folder containing the XML files
folder_path = r'C:\Development\BrickLinkReport'
# Scan the folder and use all the XML files found
xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

# Print the list of XML files found for debugging
print("XML files found:", xml_files)

# Initialize PdfPages to save the report
pdf_path = os.path.join(folder_path, 'LEGO_Set_Color_Quantities_Report.pdf')
with PdfPages(pdf_path) as pdf:
    all_warnings = []
    overall_status = []
    parts_count = []
    parts_count_owned = []
    total_min_qty = 0
    total_qty_filled = 0
    total_qty = 0
    total_qty_overall = 0

    overall_color_distribution = defaultdict(int)
    overall_color_filled_distribution = defaultdict(int)

    for xml_file in xml_files:
        # Print the full path of the current XML file for debugging
        file_path = os.path.join(folder_path, xml_file)

        # Initialize dictionaries to store the data
        min_qty_data = defaultdict(int)
        qty_filled_data = defaultdict(int)
        total_qty_data = defaultdict(int)
        warnings = []
        set_min_qty = 0
        set_qty_filled = 0

        # Parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Extract data from XML
        for item in root.findall('ITEM'):
            color_element = item.find('COLOR')
            qty_element = item.find('MINQTY')
            qty_own = item.find('QTYFILLED')
            item_id_element = item.find('ITEMID')
            item_id = item_id_element.text if item_id_element is not None else 'Unknown ID'
            
            color = color_element.text if color_element is not None and color_element.text and color_element.text.isdigit() else '0'
            min_qty = int(qty_element.text) if qty_element is not None and qty_element.text and qty_element.text.isdigit() else 0
            qty_filled = int(qty_own.text) if qty_own is not None and qty_own.text is not None and qty_own.text.isdigit() else 0
            total_qty = min_qty + qty_filled

            min_qty_data[color] += min_qty
            qty_filled_data[color] += qty_filled
            total_qty_data[color] += total_qty
            overall_color_distribution[color] += total_qty
            overall_color_filled_distribution[color] += qty_filled

            set_min_qty += min_qty
            set_qty_filled += qty_filled

        # Calculate completion status for the set
        set_completion_status = (set_qty_filled / (set_qty_filled + set_min_qty)) * 100 
        overall_status.append((xml_file, set_completion_status))
        parts_count.append((xml_file, set_qty_filled + set_min_qty))
        parts_count_owned.append((xml_file, set_qty_filled))

        total_min_qty += set_min_qty
        total_qty_filled += set_qty_filled

        # Prepare data for plotting
        colors = list(min_qty_data.keys())
        min_qty_values = [min_qty_data[color] for color in colors]
        qty_filled_values = [qty_filled_data[color] for color in colors]
        total_qty_values = [total_qty_data[color] for color in colors]

        # Sort by descending order of total_qty_values
        sorted_indices = sorted(range(len(total_qty_values)), key=lambda k: total_qty_values[k], reverse=True)
        colors = [colors[i] for i in sorted_indices]
        min_qty_values = [min_qty_values[i] for i in sorted_indices]
        qty_filled_values = [qty_filled_values[i] for i in sorted_indices]
        total_qty_values = [total_qty_values[i] for i in sorted_indices]

        # Map color codes to color names
        color_names = []
        for color in colors:
            if color in color_mapping:
                color_names.append(color_mapping[color])
            else:
                color_names.append(color)
                warning_message = f"Warning: Color code {color} not found in color mapping. Using color code as label. File: {xml_file}, Item ID: {item_id}"
                warnings.append(warning_message)
                all_warnings.append(warning_message)

        x = range(len(colors))

        # Plot the data
        plt.figure(figsize=(14, 8))
        bar_width = 0.25

        plt.bar(x, min_qty_values, width=bar_width, label='Min Qty', align='center', color='#0072B2')
        plt.bar([p + bar_width for p in x], qty_filled_values, width=bar_width, label='Qty Filled', align='center', color='#E69F00')
        plt.bar([p + bar_width * 2 for p in x], total_qty_values, width=bar_width, label='Total Qty', align='center', color='#009E73')

        plt.xlabel('Color')
        plt.ylabel('Quantity')
        plt.title(f'LEGO Set Color Quantities - {xml_file}')
        plt.xticks([p + bar_width for p in x], color_names, rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()

        # Add the pie chart for qty filled vs not filled in the top right corner
        inset_ax = plt.axes([0.65, 0.55, 0.3, 0.3])  # Position: [left, bottom, width, height]
        labels = ['Qty Filled', 'Qty Not Filled']
        sizes = [set_qty_filled, set_min_qty]
        colors = ['#ff9999','#66b3ff']
        explode = (0.1, 0)  # explode the 1st slice (Qty Filled)

        inset_ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
                     shadow=True, startangle=140)
        inset_ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        inset_ax.set_title('Completion')

        # Save the plot to the PDF file
        pdf.savefig()
        plt.close()

    # Calculate overall completion status
    overall_completion_status = (total_qty_filled / (total_qty_filled + total_min_qty)) * 100
    # Calculate total number of bricks
    total_bricks = sum(overall_color_distribution.values())
    total_bricks_owned = sum(overall_color_filled_distribution.values())
    # Add overall status to the first page of the PDF
    plt.figure(figsize=(8, 14))  # Vertical page
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
    plt.text(0.1, 0.95, f'Overall Collection Completion Status: {overall_completion_status:.2f}%', ha='left', va='top', fontsize=14, wrap=True)
    plt.text(0.1, 0.9, f'Overall number of Bricks: {total_bricks:.0f}', ha='left', va='top', fontsize=14, wrap=True)
    plt.text(0.1, 0.85, f'Overall number of Bricks Owned: {total_bricks_owned:.0f}', ha='left', va='top', fontsize=14, wrap=True)
    status_text = "\n".join([f"{xml_file}: {status:.2f}%" for xml_file, status in overall_status])
    parts_count_text = "\n".join([f"{xml_file}: {count} bricks" for xml_file, count in parts_count])
    combined_text = "\n".join([f"{xml_file}: {status:.2f}% - {count} bricks" for (xml_file, status), (_, count) in zip(overall_status, parts_count)])
    plt.text(0.1, 0.8, "Set Completion Status:\n" + combined_text, ha='left', va='top', fontsize=11, wrap=True)
    plt.axis('off')
    pdf.savefig()
    plt.close()

    # Plot overall color distribution
    overall_colors = list(overall_color_distribution.keys())
    overall_color_values = [overall_color_distribution[color] for color in overall_colors]
    overall_color_filled_values = [overall_color_filled_distribution[color] for color in overall_colors]
    overall_color_names = [color_mapping[color] if color in color_mapping else color for color in overall_colors]

    # Sort by descending order
    sorted_indices = sorted(range(len(overall_color_values)), key=lambda k: overall_color_values[k], reverse=True)
    overall_colors = [overall_colors[i] for i in sorted_indices]
    overall_color_values = [overall_color_values[i] for i in sorted_indices]
    overall_color_filled_values = [overall_color_filled_values[i] for i in sorted_indices]
    overall_color_names = [overall_color_names[i] for i in sorted_indices]

    plt.figure(figsize=(14, 8))
    bar_width = 0.35

    plt.bar(range(len(overall_colors)), overall_color_values, width=bar_width, label='Total Qty', align='center', color='#009E73')
    plt.bar([p + bar_width for p in range(len(overall_colors))], overall_color_filled_values, width=bar_width, label='Qty Filled', align='center', color='#E69F00')

    plt.xlabel('Color')
    plt.ylabel('Total Quantity')
    plt.title('Overall Color Distribution')
    plt.xticks([p + bar_width / 2 for p in range(len(overall_colors))], overall_color_names, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()

    # Add the pie chart for overall qty filled vs not filled in the top right corner
    inset_ax = plt.axes([0.65, 0.55, 0.3, 0.3])  # Position: [left, bottom, width, height]
    labels = ['Qty Filled', 'Qty Not Filled']
    sizes = [total_qty_filled, total_min_qty]
    colors = ['#ff9999','#66b3ff']
    explode = (0.1, 0)  # explode the 1st slice (Qty Filled)

    inset_ax.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
                 shadow=True, startangle=140)
    inset_ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    inset_ax.set_title('Overall Completion')

    # Save the overall color distribution plot to the PDF file
    pdf.savefig()
    plt.close()

    # Remove duplicate warnings
    all_warnings = list(set(all_warnings))

    # Add warnings to the last page of the PDF
    if all_warnings:
        plt.figure(figsize=(14, 8))  # Vertical page
        plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
        plt.text(0.5, 0.9, "Warnings:", ha='center', va='top', fontsize=14, wrap=True)
        plt.text(0.1, 0.8, "\n".join(all_warnings), ha='left', va='top', fontsize=10, wrap=True)
        plt.axis('off')
        pdf.savefig()
        plt.close()


print(f"Report saved to {pdf_path}")

In [ ]:
""" CREATING COMBINED LEGO XML FILES for ordering parts
This script processes LEGO XML inventory files to filter and combine them into a single XML file. 
It excludes specific files, filters items based on their minimum quantity, and creates both filtered 
and combined XML files.

Key Features:
1. Excludes specified XML files from processing.
2. Filters items with a minimum quantity greater than 0.
3. Combines items from multiple XML files into a single XML file, summing up quantities for duplicate items.
4. Creates filtered XML files for each input file, containing only the filtered items.

Modules Used:
- xml.etree.ElementTree: For parsing and creating XML files.
- os: For file and directory operations.

Variables:
- folder_path: Path to the folder containing the input XML files.
- filtered_folder: Path to the folder where filtered XML files will be saved.
- output_file_path: Path to the combined XML file.
- excluded_files: List of XML files to exclude from processing.
- xml_files: List of XML files to process, excluding the specified files.
- combined_root: Root element for the combined XML file.
- item_tracker: Dictionary to track existing items by (Type, Color).

Workflow:
1. Scan the folder for XML files, excluding the specified files.
2. Parse each XML file and filter items based on their minimum quantity.
3. Add filtered items to a new XML file for each input file.
4. Combine all filtered items into a single XML file, summing up quantities for duplicate items.
5. Save the filtered and combined XML files to the specified locations.

Output:
- Filtered XML files are saved in the `filtered_folder` directory with the prefix "filtered_".
- The combined XML file is saved as "wanted_list.xml" in the `filtered_folder` directory.

Usage:
- Update the `folder_path` and `filtered_folder` variables to match your directory structure.
- Add or remove file names in the `excluded_files` list as needed.
- Run the script to generate the filtered and combined XML files.
"""
import xml.etree.ElementTree as ET
import os

# Define the folder containing the XML files
folder_path = r'C:\Development\BrickLinkReport'
# Define the output folder for the filtered XML files
filtered_folder = r'C:\Development\BrickLinkReport\Filtered'
# Define the output file for the combined XML
output_file_path = os.path.join(filtered_folder, "wanted_list.xml")

# List of XML files to exclude
excluded_files = [
    "79017 - The Battle of Five Armies.xml",
    "79007 - Battle at the Black Gate.xml",
    "10333 - Barad-dur.xml",
    "10237 - Tower of Orthanc.xml"
]
# Ensure the output folder exists
os.makedirs(filtered_folder, exist_ok=True)

# Scan the folder and use all the XML files found, excluding the specified files
xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml') and f not in excluded_files]

# Print the list of XML files found for debugging
print("XML files found (excluding specified files):", xml_files)

# Create a new root for the combined XML
combined_root = ET.Element("INVENTORY")

# Dictionary to track existing items by (Type, Color)
item_tracker = {}

for xml_file in xml_files:
    # Print the full path of the current XML file for debugging
    file_path = os.path.join(folder_path, xml_file)

    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Create a new root for the filtered XML
    filtered_root = ET.Element(root.tag)

    # Extract data from XML and filter items
    for item in root.findall('ITEM'):
        qty_element = item.find('MINQTY')
        item_id_element = item.find('ITEMID')
        color_element = item.find('COLOR')

        min_qty = int(qty_element.text) if qty_element is not None and qty_element.text and qty_element.text.isdigit() else 0
        item_type = item_id_element.text if item_id_element is not None else None
        color = color_element.text if color_element is not None else None

        if min_qty > 0 and item_type and color:
            # Create a unique key for the item based on Type and Color
            item_key = (item_type, color)

            # Add to the combined file
            if item_key in item_tracker:
                # If the item already exists, add the MINQTY to the existing one
                existing_item = item_tracker[item_key]
                existing_min_qty = int(existing_item.find('MINQTY').text)
                existing_item.find('MINQTY').text = str(existing_min_qty + min_qty)
            else:
                # Create a new ITEM element for the combined file
                new_combined_item = ET.Element('ITEM')

                # Copy all sub-elements from the original item
                for sub_element in item:
                    new_sub_element = ET.SubElement(new_combined_item, sub_element.tag)
                    if sub_element.tag == 'QTYFILLED':
                        # Set QTYFILLED to 0
                        new_sub_element.text = '0'
                    else:
                        # Copy the original text
                        new_sub_element.text = sub_element.text

                # Add the REMARKS field with the name of the original file
                remarks_element = ET.SubElement(new_combined_item, 'REMARKS')
                remarks_element.text = f"{xml_file}"

                # Add the new item to the combined root and tracker
                combined_root.append(new_combined_item)
                item_tracker[item_key] = new_combined_item

            # Add to the filtered file
            new_filtered_item = ET.Element('ITEM')

            # Copy all sub-elements from the original item
            for sub_element in item:
                new_sub_element = ET.SubElement(new_filtered_item, sub_element.tag)
                if sub_element.tag == 'QTYFILLED':
                    # Set QTYFILLED to 0
                    new_sub_element.text = '0'
                else:
                    # Copy the original text
                    new_sub_element.text = sub_element.text

            # Add the new item to the filtered root
            filtered_root.append(new_filtered_item)

    # Write the filtered XML to a file
    filtered_file_path = os.path.join(filtered_folder, f"filtered_{xml_file}")
    filtered_tree = ET.ElementTree(filtered_root)
    filtered_tree.write(filtered_file_path, encoding='utf-8', xml_declaration=True)
    #print(f"Filtered XML file created: {filtered_file_path}")

# Write the combined XML to a file
combined_tree = ET.ElementTree(combined_root)
combined_tree.write(output_file_path, encoding='utf-8', xml_declaration=True)
print(f"Combined wanted list XML file created: {output_file_path}")

In [ ]:
""" CREATING THE JSON FILE FOR COLOR MAPPING
This script reads a TSV file containing color mappings and converts it into a JSON file.
the download in the txt file is necessary in case of future updates https://www.bricklink.com/catalogColors.asp
"""

import json
import os

# Percorsi dei file
INPUT_FILE = "c:\\Development\\TechnicalAnalysis\\MISC\\BL_color_mapping_FULL.txt"
OUTPUT_FILE = "c:\\Development\\TechnicalAnalysis\\MISC\\BL_color_mapping.json"

def read_tsv_to_dict(input_file):
    """Legge un file TSV e restituisce un dizionario ID -> Nome."""
    color_mapping = {}
    try:
        with open(input_file, "r", encoding="utf-8") as f:
            for line_number, line in enumerate(f, start=1):
                # Salta la riga di intestazione
                if line.startswith("ID"):
                    continue
                # Rimuovi spazi extra e dividi la riga in ID e Nome
                parts = [part.strip() for part in line.strip().split("\t") if part.strip()]
                if len(parts) == 2:
                    color_id, color_name = parts
                    color_mapping[color_id] = color_name
                else:
                    print(f"⚠️ Riga non valida al numero {line_number}: {line.strip()}")
    except FileNotFoundError:
        print(f"❌ File non trovato: {input_file}")
    except Exception as e:
        print(f"❌ Errore durante la lettura del file: {e}")
    return color_mapping

def save_dict_to_json(data, output_file):
    """Salva un dizionario in un file JSON."""
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"✅ File JSON generato con successo: {output_file}")
    except Exception as e:
        print(f"❌ Errore durante il salvataggio del file JSON: {e}")

def update_json_key(file_path, key, value):
    """Aggiorna una chiave specifica in un file JSON."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        data[key] = value
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"✅ Il valore della chiave '{key}' è stato aggiornato a '{value}' nel file {file_path}")
    except FileNotFoundError:
        print(f"❌ File non trovato: {file_path}")
    except json.JSONDecodeError:
        print(f"❌ Errore nel parsing del file JSON: {file_path}")
    except Exception as e:
        print(f"❌ Errore durante l'aggiornamento del file JSON: {e}")

def main():
    # Leggi il file TSV e crea il dizionario
    color_mapping = read_tsv_to_dict(INPUT_FILE)
    if not color_mapping:
        print("❌ Nessun dato trovato nel file di input. Verifica il formato del file.")
        return

    # Salva il dizionario in un file JSON
    save_dict_to_json(color_mapping, OUTPUT_FILE)

    # Aggiorna il valore della chiave "0" nel file JSON
    update_json_key(OUTPUT_FILE, "0", "Sticker")

if __name__ == "__main__":
    main()

In [ ]:
import requests
import csv
import xml.etree.ElementTree as ET

API_KEY = "05d2b4e46b5adb11b42169bcc0fe4745"
INPUT_CSV = "input_parts.csv"
OUTPUT_CSV = "lego_parts_list.csv"

# Mappa BrickLink → Rebrickable dei color_id
BRICKLINK_TO_REBRICKABLE_COLOR_IDS = {
    0: 0,      # Sticker / Black (placeholder)
    1: 15,     # White
    2: 19,     # Tan
    3: 14,     # Yellow
    4: 25,     # Orange
    5: 4,      # Red
    6: 2,      # Green
    7: 1,      # Blue
    11: 0,     # Black
    12: 47,    # Trans-Clear
    13: 13,    # Trans-Brown
    14: 14,    # Trans-Dark Blue
    15: 15,    # Trans-Light Blue
    16: 16,    # Trans-Neon Green
    17: 17,    # Trans-Red
    18: 18,    # Trans-Neon Orange
    19: 46,    # Trans-Yellow
    20: 34,    # Trans-Green
    21: 21,    # Chrome Gold
    28: 92,    # Nougat
    34: 27,    # Lime
    36: 10,    # Bright Green
    39: 39,    # Dark Turquoise
    42: 73,    # Medium Blue
    47: 5,     # Dark Pink
    48: 378,   # Sand Green
    55: 379,   # Sand Blue
    59: 320,   # Dark Red
    63: 272,   # Dark Blue
    65: 65,    # Metallic Gold
    67: 158,   # Yellowish Green
    68: 484,   # Dark Orange
    69: 28,    # Dark Tan
    71: 26,    # Magenta
    77: 77,    # Pearl Dark Gray
    80: 288,   # Dark Green
    85: 72,    # Dark Bluish Gray
    86: 71,    # Light Bluish Gray
    88: 70,    # Reddish Brown
    89: 85,    # Dark Purple
    90: 78,    # Light Nougat
    95: 179,   # Flat Silver
    98: 98,    # Trans-Orange
    103: 103,  # Bright Light Yellow
    104: 29,   # Bright Pink
    105: 212,  # Bright Light Blue
    108: 108,  # Trans-Bright Green
    110: 191,  # Bright Light Orange
    111: 111,  # Speckle Black-Silver
    115: 115,  # Pearl Gold
    120: 72,   # Dark Bluish Gray
    150: 150,  # Medium Nougat
    152: 152,  # Light Aqua
    153: 321,  # Dark Azure
    155: 326,  # Olive Green
    156: 322,  # Medium Azure
    158: 158,  # Yellowish Green
    159: 329,  # Glow In Dark white (Rebrickable ID approx.)
    228: 228,  # Satin Trans Clear (custom/approx)
    250: 250,  # Metallic Copper
}


# Funzione per caricare il file XML di BrickLink e estrarre i dati
def load_bricklink_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    parts = []
    for item in root.findall('ITEM'):
        item_id = item.find('ITEMID').text
        color_id = item.find('COLOR').text
        qty_element = item.find('MINQTY')
        quantity = int(qty_element.text) if qty_element is not None and qty_element.text.isdigit() else 0
        parts.append({"part_num": item_id, "color_id": color_id, "quantity": quantity})
    return parts

def convert_color_id(bl_color_id):
    return BRICKLINK_TO_REBRICKABLE_COLOR_IDS.get(bl_color_id, bl_color_id)

def get_element_from_part_color(part_num, color_id):
    url = f"https://rebrickable.com/api/v3/lego/parts/{part_num}/colors/{color_id}/"
    headers = {"Authorization": f"key {API_KEY}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        element_id = data["elements"][0] if data.get("elements") else "N/A"
        color_name = data.get("color_name", "Unknown")
        return {
            "design_id": part_num,
            "element_id": element_id,
            "color_name": color_name
        }
    else:
        print(f"[Errore] Parte {part_num}, colore {color_id} non trovata.")
        return {
            "design_id": part_num,
            "element_id": "N/A",
            "color_name": "Unknown"
        }

    
input_file = "filtered_79018 - The Lonely Mountain.xml"  # Modifica con il percorso corretto

# Carica il file XML di BrickLink
parts = load_bricklink_xml(input_file)
print (f"Caricati {len(parts)} elementi dal file XML di BrickLink.")
# Stampa i dati estratti
for part in parts:
    print(part)


# Scrittura su CSV output
with open(OUTPUT_CSV, "w", newline='', encoding="utf-8") as outfile:
    fieldnames = ["Design ID", "Element ID", "Color Name", "Quantity"]
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for part in parts:
        part_num = part["part_num"]
        color_id = int(part["color_id"])
        quantity = part["quantity"]

        reb_color = convert_color_id(color_id)
        data = get_element_from_part_color(part_num, reb_color)

        writer.writerow({
            "Design ID": data["design_id"],
            "Element ID": data["element_id"],
            "Color Name": data["color_name"],
            "Quantity": quantity
        })

        print(f"Parte {part_num}, Colore {reb_color} convertito in Element ID {data['element_id']} ({data['color_name']}) con quantità {quantity}.")
        if data["element_id"] == "N/A":
            print(f"[Errore] Element ID non trovato per parte {part_num}, colore {reb_color}.")
        else:
            print(f"Elemento trovato: {data['element_id']} ({data['color_name']})")


# Stampa un messaggio di completamento
print(f"✅ Conversione completata. File scritto: {OUTPUT_CSV}")

In [ ]:
import requests
import csv
import time

API_KEY = "05d2b4e46b5adb11b42169bcc0fe4745"
PART_NUMS = ["3002", "3023", "18653", "3710", "2780"]  # aggiungine altri se vuoi
OUTPUT_FILE = "bricklink_to_rebrickable_color_map.csv"

headers = {
    "Authorization": f"key {API_KEY}"
}

color_map = {}  # (color_name → rebrickable_color_id)

with open(OUTPUT_FILE, "w", newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Color Name", "Rebrickable Color ID", "Sample Element ID", "Part Num"])

    for part_num in PART_NUMS:
        url = f"https://rebrickable.com/api/v3/lego/parts/{part_num}/colors/"
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"[Errore] {part_num} -> {response.status_code}")
            continue

        results = response.json().get("results", [])
        for entry in results:
            color_name = entry["color_name"]
            color_id = entry["color_id"]
            elements = entry.get("elements", [])
            sample_element = elements[0] if elements else "N/A"

            if color_name not in color_map:
                color_map[color_name] = color_id
                writer.writerow([color_name, color_id, sample_element, part_num])
        
        time.sleep(1)  # rispetto rate limit API

print("✅ Mappa colori creata:")
print(color_map)
